In [1]:
from leaderboard import *

In [2]:
data = load_data(leaderboard_id=976545, year=2022)
data

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
1614144,68,7,32,61,577,24,32,16,29,277,6,7,7,235,1276,4829,917,303,<NA>
2316524,3353,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2728245,953,868,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
740740,1001,976,7452,325,47268,148,1381,68881,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
999754,434,1572,680,750,770,302,<NA>,2101,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Alex Ji,914,530,441,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
attoPascal,117,513,444,267,394,40,247,1215,1004,618,776,176,874,573,1677,5633,3164,7673,362
Azratosh,120,103,247,181,456,49,1084,950,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
benjo121ben,419,952,222,1083,325,107,1520,3241,1673,1211,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Benni S,475,788,2688,276,628,1593,417,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [3]:
points(data).head(20)

,pts,name
1,1043,1614144
2,988,attoPascal
3,937,Vonuras
4,909,TechnikTobi
5,895,Sanjay
6,774,Salominius
7,761,msonnberger
8,750,felpower
9,724,michalrzak
10,704,cluosh


In [4]:
seconds_latest_day(data)

,19
Vonuras,259
attoPascal,362


In [5]:
sum_seconds(data) // 60

,0
attoPascal,429
Vonuras,1369
